In [53]:
import pandas as pd
import os 
print(os.getcwd())
df_source = pd.read_csv("data/transcripts/whole_data_2021_2016.csv")
df_source_tot = pd.read_csv("data/transcripts/2016_full_behaviors_annotation_w_hedges.csv")

/Users/ewenmichel/Desktop/Centrale/3A/SDI/Project/Multimodal-CSC


In [54]:
columns = pd.read_csv("data/transcripts/clean_D3_S1.csv").columns
df_time = pd.DataFrame(columns = pd.read_csv("data/transcripts/clean_D3_S1.csv").columns)

for file in os.listdir("./data/transcripts/"):
    if file.startswith("clean_"):
        curr_df = pd.read_csv('./data/transcripts/' + file)
        if "Temps de départ - hh:mm:ss.ms" in curr_df.columns:
            curr_df["Begin Time - hh:mm:ss.ms"] = curr_df["Temps de départ - hh:mm:ss.ms"]
            curr_df = curr_df.drop(columns = ["Temps de départ - hh:mm:ss.ms"])
        if 'temps de fin - hh:mm:ss.ms' in curr_df.columns:
            curr_df['End Time - hh:mm:ss.ms'] = curr_df['temps de fin - hh:mm:ss.ms']
            curr_df = curr_df.drop(columns = ['temps de fin - hh:mm:ss.ms'])
        if 'Durée - hh:mm:ss.ms' in curr_df.columns:
            curr_df['Duration - hh:mm:ss.ms'] = curr_df['Durée - hh:mm:ss.ms']
            curr_df = curr_df.drop(columns = ['Durée - hh:mm:ss.ms'])
        df_time = pd.concat([df_time, curr_df])

        if '4_S2' in file:
            print(curr_df.columns)

df_time = df_time.drop(columns = ["Unnamed: 0"])
df_time = df_time[['Dyad', 'Session', 'Begin Time - hh:mm:ss.ms', 'End Time - hh:mm:ss.ms', 'Duration - hh:mm:ss.ms', 'P1', 'P2']]
print(len(df_time))

Index(['Unnamed: 0', 'Dyad', 'Session', 'P1', 'P2', 'Begin Time - hh:mm:ss.ms',
       'End Time - hh:mm:ss.ms', 'Duration - hh:mm:ss.ms'],
      dtype='object')
10134


In [75]:
df_source["Time_begin_ts"] = pd.to_datetime(df_source["Time_begin"], format = "%M:%S.%f")
df_time['Time_begin_ts'] = pd.to_datetime(df_time["Begin Time - hh:mm:ss.ms"], format = "%H:%M:%S.%f")

In [83]:
print(df_time["Time_begin_ts"].iloc[0] + pd.Timedelta(seconds = 10) < df_time["Time_begin_ts"].iloc[0])

False


In [57]:
df_source = pd.read_csv("data/transcripts/whole_data_2021_2016.csv")

sd_df = df_source[(df_source["SD_Tutor"] == "SD") | (df_source["SD_Tutee"] == "SD")].replace("SD", "x").fillna("")
qe_df = df_source[(df_source["SD_Tutor"] == "QE") | (df_source["SD_Tutee"] == "QE")].replace("QE", "x").fillna("")
sv_df = df_source[(df_source["SV_Tutor"] == "SV") | (df_source["SV_Tutee"] == "SV")].replace("SV", "x").fillna("")
pr_df = df_source[(df_source["PR_Tutor"].isin(['LPA', 'UL', 'LPP', 'LPB', '0', 'LP'])) | (df_source["PR_Tutee"].isin(['LPA', 'UL', 'LPP', 'LPB', '0', 'LP']))].replace(['LPA', 'UL', 'LPP', 'LPB', '0', 'LP'], "x").fillna("")

sd_df["SD"] = sd_df.SD_Tutor + sd_df.SD_Tutee
qe_df["QE"] = qe_df.SD_Tutor + qe_df.SD_Tutee
sv_df["SV"] = sv_df.SV_Tutor + sv_df.SV_Tutee
pr_df["PR"] = pr_df.PR_Tutor + pr_df.PR_Tutee

sd_map_dict = dict(zip(sd_df.index, sd_df["SD"]))
qe_map_dict = dict(zip(qe_df.index, qe_df["QE"]))
sv_map_dict = dict(zip(sv_df.index, sv_df["SV"]))
pr_map_dict = dict(zip(pr_df.index, pr_df["PR"]))

df_source["SD"] = df_source.index.to_series().map(sd_map_dict)
df_source["QE"] = df_source.index.to_series().map(qe_map_dict)
df_source["SV"] = df_source.index.to_series().map(sv_map_dict)
df_source["PR"] = df_source.index.to_series().map(pr_map_dict)

df_source = df_source.drop(df_source[df_source["SV"] == "xx"].index)
df_source["SD"].unique(), df_source["QE"].unique(), df_source["SV"].unique(), df_source["PR"].unique()

(array([nan, 'x'], dtype=object),
 array([nan, 'x'], dtype=object),
 array([nan, 'x'], dtype=object),
 array([nan, 'x'], dtype=object))

In [58]:
df_source.columns

Index(['Unnamed: 0', 'Dyad', 'Session', 'Period', 'P1', 'P2', 'Time_begin',
       'Time_end', 'Duration', 'SV_Tutor', 'SV_Tutee', 'PR_Tutor', 'PR_Tutee',
       'SD_Tutor', 'SD_Tutee', 'SD', 'QE', 'SV', 'PR'],
      dtype='object')

In [61]:
df_source = df_source[['Time_begin', 'Dyad', 'Session',  'P1', 'P2', 'SD', 'QE', 'SV', 'PR']]

In [60]:
df_time = df_time[['Dyad', 'Session', 'Begin Time - hh:mm:ss.ms', 'End Time - hh:mm:ss.ms', 'Duration - hh:mm:ss.ms', 'P1', 'P2']]

In [65]:
df_time["Begin Time - hh:mm:ss.ms"].iloc[0]

'00:00:12.088'

In [66]:
df_source["Time_begin"].iloc[0]

'00:01.0'

In [8]:
df_result = df_time.merge(df_source, on = ['Dyad', 'Session', 'P1', 'P2'], how = "left")
df_result = df_result.drop_duplicates().sort_values(by = ['Dyad', 'Session',  'Begin Time - hh:mm:ss.ms'])
df_result = df_result.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]

In [ ]:
    # if dyad_ == 11: 
    #     df_source_fz_ = df_source[(df_source["Dyad"] == dyad_) & (df_source["Session"] == 1)]
    #     df_time_fz_ = df_time[(df_time["Dyad"] == dyad_) & (df_time["Session"] == 1)]

    #     df_source_fz_ = df_source_fz_[(df_source_fz_[f"P1"].notna()) | (df_source_fz_[f"P2"].notna())]
    #     df_time_fz_ = df_time_fz_[(df_time_fz_[f"P1"].notna()) | (df_time_fz_[f"P2"].notna())]

    #     df_source_fz_["P1"].loc[df_source_fz_["P1"].notna()] = "1" + df_source_fz_["P1"]
    #     df_source_fz_["P2"].loc[df_source_fz_["P2"].notna()] = "2" + df_source_fz_["P2"]

    #     df_source_fz_ = df_source_fz_.fillna('')
    #     df_source_fz_["P"] = df_source_fz_["P1"] + df_source_fz_["P2"]

    #     source_P = df_source_fz_["P"].tolist()
    #     source_timestamps = df_source_fz_["Time_begin"] # mm:ss.ms

    #     for utt_, ts_ in zip(source_P, source_timestamps):
    #         ts_ = pd.to_datetime(ts_, format = "%M:%S.%f")
    #         curr_df_time_fz_ = df_time_fz_[(df_time_fz_["Time_begin_ts"] < ts_ + pd.Timedelta(seconds=10)) | (df_time_fz_["Time_begin_ts"] > ts_ - pd.Timedelta(seconds=10))]
    #         time_P1 = curr_df_time_fz_["P1"].tolist()
    #         time_P2 = curr_df_time_fz_["P2"].tolist()
    #         if utt_[0] == "1":
    #             result.append(process.extract(utt_, time_P1, limit = 2))
    #         elif utt_[0] == "2":
    #             result.append(process.extract(utt_, time_P2, limit = 2))

In [108]:
from fuzzywuzzy import fuzz, process

result = []

dyad_list = [3,4,5,6,7,8,10]
session_list = [1,2]
# person_list = [1, 2]
threshold = 10

df_source = df_source[(df_source["P1"].notna()) ^ (df_source["P2"].notna())]

for dyad_ in dyad_list:
    for session_ in session_list:
        df_source_fz_ = df_source[(df_source["Dyad"] == dyad_) & (df_source["Session"] == session_)]
        df_time_fz_ = df_time[(df_time["Dyad"] == dyad_) & (df_time["Session"] == session_)]

        df_source_fz_ = df_source_fz_[(df_source_fz_[f"P1"].notna()) | (df_source_fz_[f"P2"].notna())]
        df_time_fz_ = df_time_fz_[(df_time_fz_[f"P1"].notna()) | (df_time_fz_[f"P2"].notna())]

        df_source_fz_["P1"].loc[df_source_fz_["P1"].notna()] = "1" + df_source_fz_["P1"]
        df_source_fz_["P2"].loc[df_source_fz_["P2"].notna()] = "2" + df_source_fz_["P2"]

        df_source_fz_ = df_source_fz_.fillna('')
        df_source_fz_["P"] = df_source_fz_["P1"] + df_source_fz_["P2"]

        source_P = df_source_fz_["P"].tolist()
        source_timestamps = df_source_fz_["Time_begin"] # mm:ss.ms

        for utt_, ts_ in zip(source_P, source_timestamps):
            ts_ = pd.to_datetime(ts_, format = "%M:%S.%f")
            curr_df_time_fz_ = df_time_fz_[(df_time_fz_["Time_begin_ts"] < ts_ + pd.Timedelta(seconds=10)) | (df_time_fz_["Time_begin_ts"] > ts_ - pd.Timedelta(seconds=10))]
            time_P1 = curr_df_time_fz_["P1"].tolist()
            time_P2 = curr_df_time_fz_["P2"].tolist()
            if utt_[0] == "1":
                result.append(process.extract(utt_[1:], time_P1, limit = 2))
            elif utt_[0] == "2":
                result.append(process.extract(utt_[1:], time_P2, limit = 2))

            # df_source.loc[((df_source["Dyad"] == dyad_) & (df_source["Session"] == session_) & (df_source["P1"].notna()))]['match'] = result

df_source_result = df_source[((df_source["P1"].notna()) ^ (df_source["P2"].notna())) & (df_source["Dyad"].isin(dyad_list)) & (df_source["Session"].isin(session_list))]
df_source_result["Match"] = result
# df_source_fz = df_source[df_source["P1"].notna()]
# df_time_fz = df_source[df_source["P1"].notna()]

# source_P1 = df_source_fz["P1"].tolist()
# time_P1 = df_time_fz["P1"].tolist()

# threshold = 60

# result = []

# for utt in source_P1:
#     result.append(process.extract(utt, time_P1, limit = 3))

/var/folders/l7/1yhnrkcx2f79z5dph4rg3h540000gn/T/ipykernel_88544/3697408124.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source_fz_["P1"].loc[df_source_fz_["P1"].notna()] = "1" + df_source_fz_["P1"]
/var/folders/l7/1yhnrkcx2f79z5dph4rg3h540000gn/T/ipykernel_88544/3697408124.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source_fz_["P2"].loc[df_source_fz_["P2"].notna()] = "2" + df_source_fz_["P2"]
/var/folders/l7/1yhnrkcx2f79z5dph4rg3h540000gn/T/ipykernel_88544/3697408124.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the document

In [105]:
df_source_result.to_csv("fuzzy_test.csv")

In [12]:
import fuzzy_pandas as fpd 
df_test = fpd.fuzzy_merge(df_time, df_source,
                left_on=['Dyad', 'Session', 'P1', 'P2'],
                ignore_case=True,
                keep='match')

TypeError: can only concatenate str (not "NoneType") to str

In [226]:
df_result.to_csv("test1.csv")

In [232]:
for label in ["SD", "QE", "PR", "SV"]:
    n_result = len(df_result[df_result[label].notna()])
    n_source = len(df_source[df_source[label].notna()])
    print(f"Number of {label} in result df : {n_result}")
    print(f"Number of {label} in source df : {n_source}")

Number of SD in result df : 334
Number of SD in source df : 738
Number of QE in result df : 76
Number of QE in source df : 163
Number of PR in result df : 54
Number of PR in source df : 114
Number of SV in result df : 1246
Number of SV in source df : 1471


In [227]:
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

# compare_P1 = pd.MultiIndex.from_product([df_source['P1'], df_target['P1']]).to_series()
# compare_P2 = pd.MultiIndex.from_product([df_source['P2'], df_target['P2']]).to_series()
# compare_D = pd.MultiIndex.from_product([df_source['D'], df_target['D']]).to_series()
# compare_S = pd.MultiIndex.from_product([df_source['S'], df_target['S']]).to_series()
# compare_S = pd.MultiIndex.from_product([df_source['P1'], df_target['P1']]).to_series()
# compare_S = pd.MultiIndex.from_product([df_source['P1'], df_target['P1']]).to_series()

# comparisons = [compare_P1, compare_P2, compare_D, compare_S]

# def metrics(tup):
#     return pd.Series([fuzz.ratio(*tup),
#                       fuzz.token_sort_ratio(*tup)],
#                      ['ratio', 'token'])``

# for comp_ in comparisons:
#     comp_.apply(metrics)
#     comp_.apply(metrics).unstack().idxmax().unstack(0)
#     comp_.apply(metrics).unstack(0).idxmax().unstack(0)

SyntaxError: invalid syntax (721831928.py, line 16)